In [3]:
######### final model NN MIXED

import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-formatted_cat.csv")

df = df.drop(df.columns[0], axis=1)
df = df.drop('size_int',axis= 1)
df.columns.tolist()

['numerical_price',
 'num_bedrooms',
 'num_rooms',
 'f_string_embedding',
 'building_type_Bestaande bouw',
 'building_type_Nieuwbouw',
 'building_type_na',
 'tag_k.k.',
 'tag_v.o.n.',
 'house_category_Appartement',
 'house_category_Bungalow',
 'house_category_Eengezinswoning',
 'house_category_Grachtenpand',
 'house_category_Herenhuis',
 'house_category_Landhuis',
 'house_category_Other',
 'house_category_Unknown',
 'house_category_Villa',
 'house_category_Woonboerderij',
 'house_category_Woonboot',
 'energy_label_encoded',
 'size_scaled',
 'longitude_scaled',
 'latitude_scaled']

In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["f_string_embedding"] = df["f_string_embedding"].apply(literal_eval).apply(np.array)


x = df["f_string_embedding"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["f_string_embedding", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
import numpy as np

def train_and_evaluate_model(x_train, y_train, x_test, y_test, params):
    """
    Train and evaluate a neural network model.

    Parameters:
    - x_train: Training features
    - y_train: Training target values
    - x_test: Test features
    - y_test: Test target values
    - params: Dictionary containing the optimal parameters

    Returns:
    - R2 score, MSE, and RMSE on the test set.
    """
    x_train = x_train.to_numpy()
    x_test = x_test.to_numpy()
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')

    
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)


 
    model = Sequential()
    if params['regularization'] == 'l1':
        reg = l1(params['l1_reg'])
    elif params['regularization'] == 'l2':
        reg = l2(0)  
    elif params['regularization'] == 'l1_l2':
        reg = l1_l2(l1=params['l1_reg'], l2=0)  
    else:
        reg = None

    model.add(Dense(params['neurons_layer_1'], activation='relu', input_shape=(x_train.shape[1],), kernel_regularizer=reg))
    model.add(Dense(params['neurons_layer_2'], activation='relu', kernel_regularizer=reg))
    model.add(Dense(params['neurons_layer_3'], activation='relu', kernel_regularizer=reg))
    model.add(Dense(1, activation='linear'))

    # Compile the model
    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Scaling y_train
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()

    # Fit the model
    model.fit(x_train, y_train_scaled, epochs=params['epochs'], batch_size=params['batch_size'], verbose=1)

    # Predict and evaluate on the test set
    y_pred_scaled = model.predict(x_test)
    y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

     # Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    # R-squared Score
    r2_score_value = r2_score(y_test, y_pred)
    # Mean Absolute Error
    mae = mean_absolute_error(y_test, y_pred)
    # Mean Absolute Percentage Error
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    # Explained Variance Score
    explained_variance = explained_variance_score(y_test, y_pred)


    return r2_score_value, mse, rmse, mae, mape, explained_variance


optimal_params = {'learning_rate': 0.0008940829673560239, 'neurons_layer_1': 272, 'neurons_layer_2': 256, 'neurons_layer_3': 192, 'batch_size': 128, 'epochs': 92, 'regularization': 'none'}





In [6]:

r2_score_value, mse, rmse, mae, mape, explained_variance = train_and_evaluate_model(x_train, y_train, x_test, y_test, optimal_params)
print("R2 Score on Test Data:", r2_score_value)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("Mean Absolute percentage Error:", mape)
print("explained variance", explained_variance)

Epoch 1/92
451/451 [==============================] - 5s 8ms/step - loss: 0.3602
Epoch 2/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2853
Epoch 3/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2405
Epoch 4/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2371
Epoch 5/92
451/451 [==============================] - 4s 8ms/step - loss: 0.2010
Epoch 6/92
451/451 [==============================] - 3s 8ms/step - loss: 0.2015
Epoch 7/92
451/451 [==============================] - 3s 8ms/step - loss: 0.1761
Epoch 8/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1627
Epoch 9/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1708
Epoch 10/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1490
Epoch 11/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1437
Epoch 12/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1251
Epoch 13/92
451/451 [====

In [7]:
import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-englisch_cat.csv")

df.columns.tolist()

df = df.drop(df.columns[0], axis=1)

In [8]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["ada_embedding_eng"] = df["ada_embedding_eng"].apply(literal_eval).apply(np.array)

# Separating the target variable and embeddings
x = df["ada_embedding_eng"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["ada_embedding_eng", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)


In [9]:
r2_score_value, mse, rmse, mae, mape, explained_variance = train_and_evaluate_model(x_train, y_train, x_test, y_test, optimal_params)
print("R2 Score on Test Data:", r2_score_value)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("Mean Absolute percentage Error:", mape)
print("explained variance", explained_variance)

Epoch 1/92
451/451 [==============================] - 4s 7ms/step - loss: 0.3412
Epoch 2/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2897
Epoch 3/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2363
Epoch 4/92
451/451 [==============================] - 3s 8ms/step - loss: 0.2205
Epoch 5/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1861
Epoch 6/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1628
Epoch 7/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1521
Epoch 8/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1651
Epoch 9/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1331
Epoch 10/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1419
Epoch 11/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1280
Epoch 12/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1164
Epoch 13/92
451/451 [====

In [10]:
import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-dutch_cat.csv")

df.columns.tolist()

df = df.drop(df.columns[0], axis=1)
df = df.drop('descrip_en',axis= 1)


In [11]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["ada_embedding"] = df["ada_embedding"].apply(literal_eval).apply(np.array)

# Separating the target variable and embeddings
x = df["ada_embedding"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["ada_embedding", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)

In [12]:
r2_score_value, mse, rmse, mae, mape, explained_variance = train_and_evaluate_model(x_train, y_train, x_test, y_test, optimal_params)
print("R2 Score on Test Data:", r2_score_value)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("Mean Absolute percentage Error:", mape)
print("explained variance", explained_variance)

Epoch 1/92
451/451 [==============================] - 4s 7ms/step - loss: 0.3444
Epoch 2/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2997
Epoch 3/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2339
Epoch 4/92
451/451 [==============================] - 3s 8ms/step - loss: 0.2175
Epoch 5/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2209
Epoch 6/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1841
Epoch 7/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1608
Epoch 8/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1610
Epoch 9/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1601
Epoch 10/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1356
Epoch 11/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1252
Epoch 12/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1350
Epoch 13/92
451/451 [====

In [13]:
import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-prompt_cat.csv")


df = df.drop(df.columns[0], axis=1)
df.columns.tolist()

['numerical_price',
 'ada_embedding_prompt',
 'num_bedrooms',
 'num_rooms',
 'building_type_Bestaande bouw',
 'building_type_Nieuwbouw',
 'building_type_na',
 'tag_k.k.',
 'tag_v.o.n.',
 'house_category_Appartement',
 'house_category_Bungalow',
 'house_category_Eengezinswoning',
 'house_category_Grachtenpand',
 'house_category_Herenhuis',
 'house_category_Landhuis',
 'house_category_Other',
 'house_category_Unknown',
 'house_category_Villa',
 'house_category_Woonboerderij',
 'house_category_Woonboot',
 'energy_label_encoded',
 'size_scaled',
 'longitude_scaled',
 'latitude_scaled']

In [14]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["ada_embedding_prompt"] = df["ada_embedding_prompt"].apply(literal_eval).apply(np.array)

# Separating the target variable and embeddings
x = df["ada_embedding_prompt"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["ada_embedding_prompt", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)


In [15]:
r2_score_value, mse, rmse, mae, mape, explained_variance = train_and_evaluate_model(x_train, y_train, x_test, y_test, optimal_params)
print("R2 Score on Test Data:", r2_score_value)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("Mean Absolute percentage Error:", mape)
print("explained variance", explained_variance)

Epoch 1/92
450/450 [==============================] - 4s 6ms/step - loss: 0.3645
Epoch 2/92
450/450 [==============================] - 3s 6ms/step - loss: 0.2777
Epoch 3/92
450/450 [==============================] - 3s 7ms/step - loss: 0.2389
Epoch 4/92
450/450 [==============================] - 3s 7ms/step - loss: 0.2385
Epoch 5/92
450/450 [==============================] - 3s 8ms/step - loss: 0.2104
Epoch 6/92
450/450 [==============================] - 3s 7ms/step - loss: 0.1873
Epoch 7/92
450/450 [==============================] - 3s 8ms/step - loss: 0.1976
Epoch 8/92
450/450 [==============================] - 3s 8ms/step - loss: 0.1661
Epoch 9/92
450/450 [==============================] - 3s 7ms/step - loss: 0.1590
Epoch 10/92
450/450 [==============================] - 3s 7ms/step - loss: 0.1577
Epoch 11/92
450/450 [==============================] - 3s 7ms/step - loss: 0.1427
Epoch 12/92
450/450 [==============================] - 3s 7ms/step - loss: 0.1985
Epoch 13/92
450/450 [====

In [16]:
import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-anti_cat.csv")

df.columns.tolist()

df = df.drop(df.columns[0], axis=1)


In [17]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["description_en_anti_prompt_embedding"] = df["description_en_anti_prompt_embedding"].apply(literal_eval).apply(np.array)

# Separating the target variable and embeddings
x = df["description_en_anti_prompt_embedding"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["description_en_anti_prompt_embedding", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)


In [18]:
r2_score_value, mse, rmse, mae, mape, explained_variance = train_and_evaluate_model(x_train, y_train, x_test, y_test, optimal_params)
print("R2 Score on Test Data:", r2_score_value)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("Mean Absolute percentage Error:", mape)
print("explained variance", explained_variance)

Epoch 1/92
451/451 [==============================] - 4s 7ms/step - loss: 0.3673
Epoch 2/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2730
Epoch 3/92
451/451 [==============================] - 3s 8ms/step - loss: 0.2367
Epoch 4/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2313
Epoch 5/92
451/451 [==============================] - 3s 7ms/step - loss: 0.2013
Epoch 6/92
451/451 [==============================] - 3s 7ms/step - loss: 0.1819
Epoch 7/92
451/451 [==============================] - 3s 8ms/step - loss: 0.2165
Epoch 8/92
451/451 [==============================] - 3s 8ms/step - loss: 0.1613
Epoch 9/92
451/451 [==============================] - 4s 8ms/step - loss: 0.1560
Epoch 10/92
451/451 [==============================] - 4s 9ms/step - loss: 0.1464
Epoch 11/92
451/451 [==============================] - 4s 9ms/step - loss: 0.1338
Epoch 12/92
451/451 [==============================] - 5s 11ms/step - loss: 0.1245
Epoch 13/92
451/451 [===